# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. They are the best equipped to handle this type of work with their expertise in data analysis.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Data Analyst",
"task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
"context": "The goal is to identify trends and predict market movements for AAPL. This requires expertise in data analysis, statistical modeling, and machine learning."
}


> Entering new CrewAgentExecutor chain...
I should start by gathering up-to-date market data for AAPL to analyze using statistical modeling and machine

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
AAPL Apple Inc. 209.00 -1.65%
Summary News Research Chart Community Statistics Historical Data Profile Financials Analysis Options Holders Sustainability
We are experiencing some temporary issues. The market data on this page is currently delayed. Please bear with us as we address this and restore your personalized lists.
Earnings Tomorrow
AAPL earnings tomorrow after market close. Analysts expect EPS of $1.62 (+6.0% YoY).
NasdaqGS - Delayed Quote • USD
Apple Inc. (AAPL)
Follow Add holdings
Earnings call at 5 PM EDT
Tune into live earnings calls
Now streaming directly on quote pages.
Got it
Time to buy AAPL?
212.50 +1.29 +(0.61%)
At close: April 30 at 4:00:01 PM EDT
209.00 -3.50 (-1.65%)
After hours: April 30 at 7:59:58 PM EDT
Time to buy AAPL?
1D 5D 3.86% 1M -2.48% 6M -7.65% YTD -15.14% 1Y 24.76% 5Y 189.31% All 165,465.50%
Key Events
Mountain Advanced Chart
Loading Chart for AAPL
Previous Close 211.21


Final Answer: 
After delegating the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst using statistical modeling and machine learning, insights and alerts about significant market opportunities or threats for AAPL can be generated. This will help in identifying trends and predicting market movements for AAPL.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: After delegating the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst using statistical modeling and machine learning, insights and alerts about significant market opportunities or threats for AAPL can be generated. This will help in identifying trends and predicting market movements for AAPL.
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Develop and refine trading strategies based on the insights from the Data Analyst and user-defined risk tolerance (Medium). Consider trading preferences (Day Trading).


> Entering new CrewAgentExecut

I need to delegate work to the Trading Strategy Developer to come up with detailed execution plans for the approved trading strategies for AAPL.

Action: Delegate work to co-worker
Action Input: 
{
"coworker": "Trading Strategy Developer",
"task": "Develop detailed execution plans for Momentum, Breakout, Trend Following, and Range Trading strategies for AAPL considering current market conditions and optimal pricing.",
"context": "As per the insights provided by the Data Analyst and considering the user's risk tolerance of medium and preference for Day Trading, the potential trading strategies for AAPL have been developed: Momentum Trading Strategy, Breakout Trading Strategy, Trend Following Strategy, and Range Trading Strategy."
}


> Entering new CrewAgentExecutor chain...
I need to gather information on Momentum, Breakout, Trend Following, and Range Trading strategies for AAPL to develop detailed execution plans.

Action: Search the internet
Action Input: {"search_query": "Momentum T

Thought: I have gathered information on Momentum, Breakout, and Trend Following Trading Strategies for AAPL. Now, I need to search for Range Trading Strategy.

Action: Search the internet
Action Input: {"search_query": "Range Trading Strategy for AAPL"} 


Search results: Title: Ideas In-Focus: If AAPL Stays Range-Bound Between ±2% Th..
Link: https://marketchameleon.com/articles/i/2025/4/4/24390-if-aapl-stays-range-bound-between-2pct-this-strateg
Snippet: If AAPL Stays Range-Bound Between ±2% This Strategy Will Yield 153% ; Buy, 182.5 Put ; Sell, 190 Put ; Sell, 197.5 Call ; Buy, 205 Call ; Credit: ...
---
Title: Day Trading Strategy for AAPL All Time High - YouTube
Link: https://www.youtube.com/watch?v=RuKmgQg7UIk
Snippet: Join Our Live Trading Community for $1: https://bearbulltraders.com/youtube Learn the Basics of Trading | Ask your Trading Questions | Get ...
---
Title: Ideas In-Focus: If AAPL Stays Range-Bound Between ±3% Th..
Link: https://marketchameleon.com/articles/i/2025/4/1

I need to thoroughly assess the risks associated with each proposed trading strategy for AAPL and provide detailed analysis and mitigation strategies to ensure alignment with the firm's risk tolerance.

Action: Read website content
Action Input: {"website_url": "AAPLtradingstrategies.com"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'AAPLtradingstrategies.com': No scheme supplied. Perhaps you meant https://AAPLtradingstrategies.com?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: I need to find information on the proposed trading strategies for AAPL to evaluate the risks associated with each one.
Action: Search the internet
Action Input: {"search_query": "AAPL trading strategies Momentum Breakout Trend Following Range"} 


Search results: Title: Momentum Trading: Types, Strategies, and More - QuantInsti Blog
Link: https://blog.quantinst

Final Answer: 

Risk Analysis Report for AAPL Trading Strategies:

Momentum Trading Strategy:
- Risk: The risk associated with the momentum trading strategy for AAPL lies in potential rapid price changes in the opposite direction, leading to losses if the momentum suddenly reverses.
- Mitigation Strategy: To mitigate this risk, it is essential to set strict stop-loss orders and closely monitor momentum indicators to detect any signs of trend reversal early.

Breakout Trading Strategy:
- Risk: The risk of the breakout trading strategy for AAPL includes false breakouts or whipsaws, where the price briefly breaks out but then quickly reverses.
- Mitigation Strategy: Traders should wait for confirmation of the breakout and set stop-loss orders at strategic levels to limit losses in case of a false breakout.

Trend Following Strategy:
- Risk: Trend following strategies for AAPL involve identifying and following the direction of the prevailing trend, with the risk of sudden trend reversals l

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Momentum Trading Strategy:
- Risk: The risk associated with the momentum trading strategy for AAPL lies in potential rapid price changes in the opposite direction, leading to losses if the momentum suddenly reverses.
- Mitigation Strategy: To mitigate this risk, it is essential to set strict stop-loss orders and closely monitor momentum indicators to detect any signs of trend reversal early.

Breakout Trading Strategy:
- Risk: The risk of the breakout trading strategy for AAPL includes false breakouts or whipsaws, where the price briefly breaks out but then quickly reverses.
- Mitigation Strategy: Traders should wait for confirmation of the breakout and set stop-loss orders at strategic levels to limit losses in case of a false breakout.

Trend Following Strategy:
- Risk: Trend following strategies for AAPL involve identifying and following the direction of the prevailing trend, with the risk of sudden trend reversals leading to losses.
- Mitigation Strategy: Traders can use trend indicators and moving averages to confirm the strength of the trend before entering a position and implement trailing stop-loss orders to protect profits.

Range Trading Strategy:
- Risk: Range trading for AAPL involves buying at the bottom of a range and selling at the top, with the risk of unexpected breakouts leading to losses.
- Mitigation Strategy: Traders can use range-bound indicators to accurately identify range boundaries and set stop-loss orders outside the range limits to limit losses in case of a breakout.

By implementing the suggested mitigation strategies for each trading strategy, traders can better manage the associated risks and improve their overall trading performance with AAPL.